### Import Module

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dense,Activation, LSTM, Dropout, TimeDistributed, Flatten, GRU
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from sklearn.utils import class_weight
from sklearn.utils import class_weight
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from collections import Counter

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten

### Data Preparation

In [3]:
training_load = pd.read_csv('training_1961_2016.csv', delimiter=',')
testing_2017_load = pd.read_csv('testing_2017.csv', delimiter=',')
testing_2018_load = pd.read_csv('testing_2018.csv', delimiter=',')
testing_2019_load = pd.read_csv('testing_2019.csv', delimiter=',')

In [4]:
training_load.head(5)

,FirstYear_index,FirstYear_playerID,FirstYear_yearID,FirstYear_teamID,FirstYear_lgID,FirstYear_stint,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,...,SixthYear_HBP,SixthYear_SH,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age
0,41261,aaronha01,1961,ML1,NL,1.0,155.0,603.0,115.0,197.0,...,1.0,0.0,8.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,32.0
1,41959,aaronha01,1962,ML1,NL,1.0,156.0,592.0,127.0,191.0,...,0.0,0.0,6.0,11.0,Hank,Aaron,Henry Louis,180.0,72.0,33.0
2,42719,aaronha01,1963,ML1,NL,1.0,161.0,631.0,121.0,201.0,...,1.0,0.0,5.0,21.0,Hank,Aaron,Henry Louis,180.0,72.0,34.0
3,43471,aaronha01,1964,ML1,NL,1.0,145.0,570.0,103.0,187.0,...,2.0,0.0,3.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,35.0
4,44225,aaronha01,1965,ML1,NL,1.0,150.0,570.0,109.0,181.0,...,2.0,0.0,6.0,13.0,Hank,Aaron,Henry Louis,180.0,72.0,36.0


In [5]:
var_name = ['index', 'playerID', 'teamID', 'lgID', 'stint','nameFirst', 'nameLast', 'nameGiven']

First = []
for i in range(len(var_name)):
    First.append(f"FirstYear_{var_name[i]}")

Second = []
for i in range(len(var_name)):
    Second.append(f"SecondYear_{var_name[i]}")

Third = []
for i in range(len(var_name)):
    Third.append(f"ThirdYear_{var_name[i]}")
    
Fourth = []
for i in range(len(var_name)):
    Fourth.append(f"FourthYear_{var_name[i]}")
    
Fifth = []
for i in range(len(var_name)):
    Fifth.append(f"FifthYear_{var_name[i]}")
    
FileColumnName = First + Second + Third + Fourth + Fifth

In [6]:
x_train = training_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(training_load), 5, 21))
y_train = training_load['SixthYear_HR'].values
x_test_2017 = testing_2017_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2017_load), 5, 21))
y_test_2017 = testing_2017_load['SixthYear_HR'].values
x_test_2018 = testing_2018_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2018_load), 5, 21))
y_test_2018 = testing_2018_load['SixthYear_HR'].values
x_test_2019 = testing_2019_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2019_load), 5, 21))
y_test_2019 = testing_2019_load['SixthYear_HR'].values

In [7]:
x_train_2018 = np.concatenate((x_train, x_test_2017, x_test_2018))
y_train_2018 = np.concatenate((y_train, y_test_2017, y_test_2018))

In [8]:
testing_2019 = testing_2019_load.drop(FileColumnName, axis=1).copy()

### Models

In [11]:
model_Hybrid = Sequential()

# CNN layers
model_Hybrid.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(5, 21), padding='same'))
model_Hybrid.add(MaxPooling1D(pool_size=2))
model_Hybrid.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model_Hybrid.add(MaxPooling1D(pool_size=2))

# LSTM layers
model_Hybrid.add(LSTM(128, return_sequences=True))
model_Hybrid.add(LSTM(64))

# Fully connected layers
model_Hybrid.add(Dense(64, activation='relu'))
model_Hybrid.add(Dropout(0.5))
model_Hybrid.add(Dense(1, activation='relu'))

# Compile the model
adam = optimizers.Adam(learning_rate=0.001)
model_Hybrid.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

# Print the model summary
model_Hybrid.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 5, 64)             4096      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 2, 64)            0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 2, 128)            24704     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 1, 128)           0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 1, 128)            131584    
                                                                 
 lstm_1 (LSTM)               (None, 64)               

2024-03-28 13:59:38.180832: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-28 13:59:38.182082: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-28 13:59:38.182755: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [8]:
model_A = Sequential()
model_A.add(GRU(128, input_shape=(5, 21), return_sequences=True))
model_A.add(GRU(128, return_sequences=True))
model_A.add(Dropout(rate=0.5))
model_A.add(Flatten())
model_A.add(Dense(1024, activation="relu"))
model_A.add(Dropout(rate=0.5))
model_A.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate = 0.001)
model_A.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_A.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 5, 128)            57984     
                                                                 
 gru_1 (GRU)                 (None, 5, 128)            99072     
                                                                 
 dropout (Dropout)           (None, 5, 128)            0         
                                                                 
 flatten (Flatten)           (None, 640)               0         
                                                                 
 dense (Dense)               (None, 1024)              656384    
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1

2024-03-19 09:24:37.222170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:37.222607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:37.223537: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [9]:
model_B = Sequential()
model_B.add(GRU(32, input_shape=(5, 21), return_sequences=True))
model_B.add(GRU(16, return_sequences=True))
model_B.add(GRU(8, return_sequences=True))
model_B.add(Dropout(rate=0.5))
model_B.add(Flatten())
model_B.add(Dense(512, activation="relu"))
model_B.add(Dropout(rate=0.5))
model_B.add(Dense(64, activation="relu"))
model_B.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate = 0.001)
model_B.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_B.summary()

2024-03-19 09:24:37.397010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:37.397657: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:37.398423: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 5, 32)             5280      
                                                                 
 gru_3 (GRU)                 (None, 5, 16)             2400      
                                                                 
 gru_4 (GRU)                 (None, 5, 8)              624       
                                                                 
 dropout_2 (Dropout)         (None, 5, 8)              0         
                                                                 
 flatten_1 (Flatten)         (None, 40)                0         
                                                                 
 dense_2 (Dense)             (None, 512)               20992     
                                                                 
 dropout_3 (Dropout)         (None, 512)              

2024-03-19 09:24:37.469472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:37.469861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:37.470373: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [10]:
model_C = Sequential()
model_C.add(GRU(32, input_shape=(5, 21), return_sequences=True))
model_C.add(GRU(32, return_sequences=True))
model_C.add(BatchNormalization())
model_C.add(Flatten())
model_C.add(Dense(512, activation="relu"))
model_C.add(BatchNormalization())
model_C.add(Dense(64, activation="relu"))
model_C.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate= 0.001)
model_C.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_C.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_5 (GRU)                 (None, 5, 32)             5280      
                                                                 
 gru_6 (GRU)                 (None, 5, 32)             6336      
                                                                 
 batch_normalization (BatchN  (None, 5, 32)            128       
 ormalization)                                                   
                                                                 
 flatten_2 (Flatten)         (None, 160)               0         
                                                                 
 dense_5 (Dense)             (None, 512)               82432     
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

2024-03-19 09:24:37.715838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:37.716540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:37.716982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [11]:
model_D = Sequential()
model_D.add(GRU(64, input_shape=(5, 21), return_sequences=True))
model_D.add(GRU(64, return_sequences=True))
model_D.add(Dropout(rate=0.5))
model_D.add(TimeDistributed(Dense(10)))
model_D.add(Flatten())
model_D.add(Dense(512, activation="relu"))
model_D.add(Dropout(rate=0.5))
model_D.add(Dense(64, activation="relu"))
model_D.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate = 0.001)
model_D.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_D.summary()

2024-03-19 09:24:37.893431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:37.894227: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:37.894652: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_7 (GRU)                 (None, 5, 64)             16704     
                                                                 
 gru_8 (GRU)                 (None, 5, 64)             24960     
                                                                 
 dropout_4 (Dropout)         (None, 5, 64)             0         
                                                                 
 time_distributed (TimeDistr  (None, 5, 10)            650       
 ibuted)                                                         
                                                                 
 flatten_3 (Flatten)         (None, 50)                0         
                                                                 
 dense_9 (Dense)             (None, 512)               26112     
                                                      

2024-03-19 09:24:37.959354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:37.959757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:37.960714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [12]:
model_E = Sequential()
model_E.add(GRU(32, input_shape=(5, 21), return_sequences=True))
model_E.add(GRU(32, return_sequences=True))
model_E.add(Dropout(rate=0.5))
model_E.add(Flatten())
model_E.add(Dense(512, activation="relu"))
model_E.add(Dropout(rate=0.5))
model_E.add(Dense(64, activation="relu"))
model_E.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate = 0.001)
model_E.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_E.summary()

2024-03-19 09:24:38.068361: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:38.068918: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:38.069465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_9 (GRU)                 (None, 5, 32)             5280      
                                                                 
 gru_10 (GRU)                (None, 5, 32)             6336      
                                                                 
 dropout_6 (Dropout)         (None, 5, 32)             0         
                                                                 
 flatten_4 (Flatten)         (None, 160)               0         
                                                                 
 dense_12 (Dense)            (None, 512)               82432     
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 64)               

2024-03-19 09:24:38.133975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:38.134498: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:38.135122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

                                                                 
Total params: 126,945
Trainable params: 126,945
Non-trainable params: 0
_________________________________________________________________


### Training

In [13]:
from sklearn.utils import class_weight

# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "Hybrid/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model_Hybrid_stat = model_Hybrid.fit(x_train_2018, y_train_2018, 
                                     batch_size=128, 
                                     epochs=1000, 
                                     validation_data=(x_test_2019, y_test_2019),
                                     callbacks=[checkpoint],
                                     shuffle=True,
                                     class_weight=adjusted_class_weights
                                    )

Epoch 1/1000
74/79 [===========================>..] - ETA: 0s - loss: 505.0352 - accuracy: 0.0892
Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to Hybrid/saved-model-001-0.0524.hdf5
79/79 [==============================] - 1s 9ms/step - loss: 490.0213 - accuracy: 0.0896 - val_loss: 371.2174 - val_accuracy: 0.0524
Epoch 2/1000
76/79 [===========================>..] - ETA: 0s - loss: 451.5362 - accuracy: 0.0902
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 8ms/step - loss: 449.0241 - accuracy: 0.0896 - val_loss: 366.1346 - val_accuracy: 0.0524
Epoch 3/1000
77/79 [============================>.] - ETA: 0s - loss: 423.4777 - accuracy: 0.0897
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 9ms/step - loss: 419.8467 - accuracy: 0.0896 - val_loss: 342.0277 - val_accuracy: 0.0524
Epoch 4/1000
77/79 [============================>.] - ETA: 0s - loss: 437.8698 - accuracy: 0.0900
Epo

In [13]:
from sklearn.utils import class_weight

# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "A/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_A_stat = model_A.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-19 09:24:38.203712: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-19 09:24:38.295158: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:38.295805: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:38.296540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG IN

78/79 [============================>.] - ETA: 0s - loss: 510.4788 - accuracy: 0.0897

2024-03-19 09:24:41.251321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:24:41.251944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:24:41.252669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to A/saved-model-001-0.0524.hdf5
79/79 [==============================] - 3s 24ms/step - loss: 509.4941 - accuracy: 0.0896 - val_loss: 449.4427 - val_accuracy: 0.0524
Epoch 2/1000
77/79 [============================>.] - ETA: 0s - loss: 464.4834 - accuracy: 0.0902
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 2s 22ms/step - loss: 460.2996 - accuracy: 0.0896 - val_loss: 367.5218 - val_accuracy: 0.0524
Epoch 3/1000
78/79 [============================>.] - ETA: 0s - loss: 435.3333 - accuracy: 0.0897
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 2s 22ms/step - loss: 434.5698 - accuracy: 0.0896 - val_loss: 288.3584 - val_accuracy: 0.0524
Epoch 4/1000
79/79 [==============================] - ETA: 0s - loss: 415.4642 - accuracy: 0.0896
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 2s 21ms/st

In [14]:
from sklearn.utils import class_weight
# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "B/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_B_stat = model_B.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-19 09:54:24.262225: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:54:24.262810: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:54:24.263849: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

76/79 [===========================>..] - ETA: 0s - loss: 636.9628 - accuracy: 0.0907

2024-03-19 09:54:26.937751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 09:54:26.938206: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 09:54:26.938981: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to B/saved-model-001-0.0524.hdf5
79/79 [==============================] - 3s 13ms/step - loss: 651.1812 - accuracy: 0.0895 - val_loss: 469.4644 - val_accuracy: 0.0524
Epoch 2/1000
73/79 [==========================>...] - ETA: 0s - loss: 412.9060 - accuracy: 0.0909
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 412.8285 - accuracy: 0.0896 - val_loss: 367.3486 - val_accuracy: 0.0524
Epoch 3/1000
73/79 [==========================>...] - ETA: 0s - loss: 430.0428 - accuracy: 0.0901
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 434.5074 - accuracy: 0.0896 - val_loss: 590.8110 - val_accuracy: 0.0524
Epoch 4/1000
73/79 [==========================>...] - ETA: 0s - loss: 495.3264 - accuracy: 0.0893
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step 

In [15]:
from sklearn.utils import class_weight
# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "C/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_C_stat = model_C.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-19 10:04:07.537653: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 10:04:07.538667: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 10:04:07.539263: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

79/79 [==============================] - ETA: 0s - loss: 655.9218 - accuracy: 0.0896

2024-03-19 10:04:09.542187: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 10:04:09.542790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 10:04:09.543378: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to C/saved-model-001-0.0524.hdf5
79/79 [==============================] - 2s 11ms/step - loss: 655.9218 - accuracy: 0.0896 - val_loss: 289.3406 - val_accuracy: 0.0524
Epoch 2/1000
77/79 [============================>.] - ETA: 0s - loss: 472.1758 - accuracy: 0.0894
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 467.8376 - accuracy: 0.0895 - val_loss: 130.9052 - val_accuracy: 0.0524
Epoch 3/1000
78/79 [============================>.] - ETA: 0s - loss: 398.1241 - accuracy: 0.0897
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 397.6352 - accuracy: 0.0896 - val_loss: 130.5477 - val_accuracy: 0.0524
Epoch 4/1000
79/79 [==============================] - ETA: 0s - loss: 449.4972 - accuracy: 0.0896
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step 

In [16]:
from sklearn.utils import class_weight
# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "D/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_D_stat = model_D.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-19 10:13:45.501674: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 10:13:45.502388: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 10:13:45.503296: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

76/79 [===========================>..] - ETA: 0s - loss: 511.1576 - accuracy: 0.0882

2024-03-19 10:13:47.821896: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 10:13:47.822330: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 10:13:47.822949: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to D/saved-model-001-0.0524.hdf5
79/79 [==============================] - 3s 14ms/step - loss: 508.8684 - accuracy: 0.0888 - val_loss: 338.1430 - val_accuracy: 0.0524
Epoch 2/1000
78/79 [============================>.] - ETA: 0s - loss: 473.3242 - accuracy: 0.0894
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 11ms/step - loss: 472.5938 - accuracy: 0.0896 - val_loss: 477.7939 - val_accuracy: 0.0524
Epoch 3/1000
76/79 [===========================>..] - ETA: 0s - loss: 453.6413 - accuracy: 0.0892
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 11ms/step - loss: 452.2611 - accuracy: 0.0896 - val_loss: 136.7254 - val_accuracy: 0.0524
Epoch 4/1000
76/79 [===========================>..] - ETA: 0s - loss: 417.5665 - accuracy: 0.0895
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 11ms/st

In [17]:
from sklearn.utils import class_weight
# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "E/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_E_stat = model_E.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-19 10:33:01.414215: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 10:33:01.414820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 10:33:01.415917: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

79/79 [==============================] - ETA: 0s - loss: 565.0131 - accuracy: 0.0890

2024-03-19 10:33:03.316349: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-19 10:33:03.316934: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-19 10:33:03.317634: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to E/saved-model-001-0.0524.hdf5
79/79 [==============================] - 2s 10ms/step - loss: 565.0131 - accuracy: 0.0890 - val_loss: 461.7191 - val_accuracy: 0.0524
Epoch 2/1000
74/79 [===========================>..] - ETA: 0s - loss: 364.7409 - accuracy: 0.0887
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 359.4034 - accuracy: 0.0896 - val_loss: 405.4883 - val_accuracy: 0.0524
Epoch 3/1000
74/79 [===========================>..] - ETA: 0s - loss: 410.2706 - accuracy: 0.0896
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 447.1553 - accuracy: 0.0896 - val_loss: 869.5049 - val_accuracy: 0.0524
Epoch 4/1000
73/79 [==========================>...] - ETA: 0s - loss: 492.0828 - accuracy: 0.0895
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step 

### Analysis

In [18]:
AAA = load_model('2019/saved-model-851-0.0942-A.hdf5')
BBB = load_model('2019/saved-model-637-0.1257-B.hdf5')
CCC = load_model('2019/saved-model-203-0.0995-C.hdf5')
DDD = load_model('2019/saved-model-109-0.1099-D.hdf5')
EEE = load_model('2019/saved-model-714-0.1152-E.hdf5')
HHH = load_model('Hybrid/saved-model-001-0.0524.hdf5')

2024-03-30 20:18:39.774947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-30 20:18:39.775554: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-30 20:18:39.776166: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [19]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = HHH.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

2024-03-30 20:18:40.042741: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-30 20:18:40.043766: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-30 20:18:40.044244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

6/6 [==============================] - 0s 2ms/step
correct answer：0.0105
plus or minus 1：0.0262
plus or minus 3：0.0942
plus or minus 5：0.1414
plus or minus 10：0.2723


In [20]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 17.095468021812238
mean_squared_error: 371.2174745752206
rmse: 19.26700481588201


In [21]:
Counter(sorted(p))

Counter({31.0: 2})

In [22]:
Counter(sorted(n))

Counter({31.0: 2, 32.0: 2, 30.0: 1})

In [23]:
Counter(sorted(m))

Counter({33.0: 4, 34.0: 4, 29.0: 3, 28.0: 2, 31.0: 2, 32.0: 2, 30.0: 1})

In [24]:
Counter(sorted(q))

Counter({12.0: 15,
         2.0: 14,
         1.0: 10,
         9.0: 9,
         3.0: 8,
         11.0: 8,
         17.0: 7,
         5.0: 6,
         6.0: 6,
         7.0: 6,
         10.0: 5,
         13.0: 5,
         15.0: 5,
         20.0: 5,
         0.0: 4,
         8.0: 4,
         14.0: 4,
         16.0: 4,
         19.0: 4,
         4.0: 3,
         18.0: 3,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [25]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [6, 79]
1: [6, 14, 79, 111, 133]
3: [0, 6, 10, 13, 14, 15, 24, 53, 65, 68, 76, 79, 111, 133, 135, 155, 162, 166]
NA: [1, 2, 3, 4, 5, 7, 9, 11, 12, 16, 20, 21, 22, 23, 25, 26, 27, 29, 30, 32, 33, 35, 36, 39, 40, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 55, 56, 57, 58, 59, 62, 64, 66, 67, 69, 70, 71, 72, 73, 74, 75, 77, 78, 80, 81, 82, 84, 85, 86, 87, 88, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110, 112, 113, 114, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 130, 131, 132, 134, 136, 137, 138, 139, 140, 142, 143, 145, 146, 147, 148, 151, 153, 154, 156, 157, 159, 160, 161, 165, 167, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 181, 183, 184, 185, 186, 187, 188, 189, 190]


In [26]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = AAA.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.0942
plus or minus 1：0.1675
plus or minus 3：0.3560
plus or minus 5：0.5183
plus or minus 10：0.7592


In [27]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.721404764664735
mean_squared_error: 77.85735240387437
rmse: 8.823681340793897


In [21]:
Counter(sorted(p))

Counter({7.0: 3,
         2.0: 2,
         12.0: 2,
         13.0: 2,
         4.0: 1,
         5.0: 1,
         6.0: 1,
         9.0: 1,
         10.0: 1,
         14.0: 1,
         20.0: 1,
         34.0: 1,
         35.0: 1})

In [22]:
Counter(sorted(n))

Counter({2.0: 3,
         3.0: 3,
         7.0: 3,
         12.0: 3,
         5.0: 2,
         6.0: 2,
         9.0: 2,
         13.0: 2,
         34.0: 2,
         1.0: 1,
         4.0: 1,
         10.0: 1,
         11.0: 1,
         14.0: 1,
         16.0: 1,
         17.0: 1,
         20.0: 1,
         25.0: 1,
         35.0: 1})

In [23]:
Counter(sorted(m))

Counter({12.0: 7,
         5.0: 6,
         2.0: 5,
         7.0: 5,
         9.0: 5,
         1.0: 4,
         3.0: 3,
         6.0: 3,
         0.0: 2,
         8.0: 2,
         13.0: 2,
         14.0: 2,
         15.0: 2,
         20.0: 2,
         23.0: 2,
         29.0: 2,
         34.0: 2,
         4.0: 1,
         10.0: 1,
         11.0: 1,
         16.0: 1,
         17.0: 1,
         19.0: 1,
         21.0: 1,
         22.0: 1,
         25.0: 1,
         27.0: 1,
         32.0: 1,
         35.0: 1})

In [24]:
Counter(sorted(q))

Counter({2.0: 4,
         1.0: 3,
         6.0: 3,
         3.0: 2,
         10.0: 2,
         12.0: 2,
         16.0: 2,
         17.0: 2,
         21.0: 2,
         23.0: 2,
         24.0: 2,
         33.0: 2,
         0.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         18.0: 1,
         19.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         31.0: 1,
         34.0: 1,
         35.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         44.0: 1,
         48.0: 1,
         49.0: 1})

In [25]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [21, 25, 57, 64, 68, 69, 80, 83, 98, 114, 119, 125, 130, 136, 142, 153, 159, 177]
1: [4, 12, 21, 25, 29, 36, 37, 57, 64, 67, 68, 69, 74, 78, 80, 83, 91, 97, 98, 108, 114, 117, 118, 119, 125, 130, 136, 142, 153, 155, 159, 177]
3: [2, 4, 10, 12, 13, 21, 22, 25, 27, 28, 29, 36, 37, 42, 57, 60, 64, 67, 68, 69, 72, 74, 75, 78, 80, 82, 83, 88, 91, 92, 93, 94, 95, 96, 97, 98, 102, 104, 108, 109, 110, 111, 114, 117, 118, 119, 121, 123, 125, 127, 130, 136, 137, 142, 147, 148, 153, 154, 155, 157, 159, 160, 163, 164, 165, 167, 177, 183]
NA: [6, 16, 24, 34, 35, 41, 46, 47, 49, 50, 54, 61, 63, 65, 66, 71, 77, 89, 90, 99, 100, 105, 106, 107, 122, 126, 128, 139, 141, 144, 145, 151, 162, 166, 169, 170, 172, 173, 174, 176, 178, 180, 181, 187, 188, 190]


In [26]:
AA = testing_2019.copy()
AA['Prediction'] = np.round(predictions)
AA['Difference'] = AA.SixthYear_HR - AA.Prediction

In [27]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = BBB.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct：0.1257
plus or minus 1：0.2094
plus or minus 3：0.3822
plus or minus 5：0.5393
plus or minus 10：0.7696


In [28]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.392659028787263
mean_squared_error: 73.43778988846655
rmse: 8.569585164316097


In [29]:
Counter(sorted(p))

Counter({12.0: 4,
         3.0: 3,
         7.0: 2,
         15.0: 2,
         18.0: 2,
         5.0: 1,
         8.0: 1,
         9.0: 1,
         10.0: 1,
         11.0: 1,
         13.0: 1,
         14.0: 1,
         16.0: 1,
         17.0: 1,
         27.0: 1,
         29.0: 1})

In [30]:
Counter(sorted(m))

Counter({12.0: 8,
         9.0: 7,
         11.0: 6,
         5.0: 5,
         2.0: 4,
         7.0: 4,
         13.0: 4,
         1.0: 3,
         3.0: 3,
         8.0: 3,
         14.0: 3,
         17.0: 3,
         20.0: 3,
         6.0: 2,
         10.0: 2,
         15.0: 2,
         18.0: 2,
         0.0: 1,
         4.0: 1,
         16.0: 1,
         21.0: 1,
         27.0: 1,
         29.0: 1,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [31]:
Counter(sorted(n))

Counter({12.0: 5,
         3.0: 3,
         9.0: 3,
         13.0: 3,
         2.0: 2,
         5.0: 2,
         7.0: 2,
         10.0: 2,
         11.0: 2,
         14.0: 2,
         15.0: 2,
         17.0: 2,
         18.0: 2,
         6.0: 1,
         8.0: 1,
         16.0: 1,
         20.0: 1,
         21.0: 1,
         27.0: 1,
         29.0: 1,
         32.0: 1})

In [32]:
Counter(sorted(q))

Counter({1.0: 3,
         2.0: 3,
         24.0: 3,
         33.0: 3,
         3.0: 2,
         4.0: 2,
         6.0: 2,
         21.0: 2,
         31.0: 2,
         34.0: 2,
         35.0: 2,
         0.0: 1,
         12.0: 1,
         15.0: 1,
         16.0: 1,
         17.0: 1,
         19.0: 1,
         23.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         41.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [33]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [13, 20, 21, 25, 29, 36, 57, 59, 72, 74, 78, 91, 97, 108, 110, 113, 121, 123, 131, 146, 153, 158, 165, 177]
1: [9, 12, 13, 20, 21, 25, 29, 36, 44, 57, 59, 60, 67, 69, 72, 74, 78, 80, 91, 97, 101, 104, 108, 110, 111, 113, 119, 121, 123, 124, 125, 130, 131, 137, 146, 153, 158, 165, 177, 185]
3: [2, 4, 7, 9, 11, 12, 13, 20, 21, 22, 23, 25, 27, 29, 36, 42, 44, 48, 55, 56, 57, 59, 60, 64, 67, 68, 69, 72, 73, 74, 75, 78, 80, 83, 86, 91, 95, 96, 97, 101, 102, 103, 104, 108, 110, 111, 113, 114, 117, 118, 119, 120, 121, 123, 124, 125, 130, 131, 137, 142, 146, 147, 148, 153, 157, 158, 159, 160, 165, 167, 177, 184, 185]
NA: [6, 15, 24, 26, 34, 35, 38, 46, 47, 49, 54, 61, 63, 65, 66, 71, 79, 89, 99, 100, 105, 106, 107, 126, 128, 129, 141, 144, 145, 155, 156, 162, 166, 169, 170, 172, 173, 174, 178, 179, 180, 181, 182, 187]


In [34]:
BB = testing_2019.copy()
BB['Prediction'] = np.round(predictions)
BB['Difference'] = BB.SixthYear_HR - BB.Prediction

In [35]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = CCC.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.0995
plus or minus 1：0.2147
plus or minus 3：0.4031
plus or minus 5：0.5602
plus or minus 10：0.7958


In [36]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.138256144149141
mean_squared_error: 68.37441066441625
rmse: 8.2688820686001


In [37]:
Counter(sorted(p))

Counter({12.0: 5,
         2.0: 3,
         7.0: 2,
         8.0: 2,
         4.0: 1,
         6.0: 1,
         17.0: 1,
         20.0: 1,
         21.0: 1,
         23.0: 1,
         24.0: 1})

In [38]:
Counter(sorted(n))

Counter({12.0: 6,
         2.0: 5,
         3.0: 3,
         7.0: 2,
         8.0: 2,
         9.0: 2,
         10.0: 2,
         15.0: 2,
         17.0: 2,
         18.0: 2,
         1.0: 1,
         4.0: 1,
         6.0: 1,
         11.0: 1,
         13.0: 1,
         14.0: 1,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         23.0: 1,
         24.0: 1,
         27.0: 1,
         32.0: 1})

In [39]:
Counter(sorted(m))

Counter({12.0: 8,
         2.0: 6,
         5.0: 6,
         11.0: 5,
         1.0: 4,
         3.0: 4,
         9.0: 4,
         6.0: 3,
         7.0: 3,
         10.0: 3,
         14.0: 3,
         22.0: 3,
         0.0: 2,
         8.0: 2,
         15.0: 2,
         16.0: 2,
         17.0: 2,
         18.0: 2,
         20.0: 2,
         23.0: 2,
         4.0: 1,
         13.0: 1,
         21.0: 1,
         24.0: 1,
         27.0: 1,
         29.0: 1,
         31.0: 1,
         32.0: 1,
         34.0: 1})

In [40]:
Counter(sorted(q))

Counter({2.0: 4,
         1.0: 3,
         24.0: 3,
         33.0: 3,
         3.0: 2,
         35.0: 2,
         0.0: 1,
         4.0: 1,
         6.0: 1,
         9.0: 1,
         10.0: 1,
         12.0: 1,
         15.0: 1,
         16.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1,
         23.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         31.0: 1,
         37.0: 1,
         39.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [41]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [7, 12, 21, 29, 43, 57, 60, 64, 67, 75, 80, 110, 119, 149, 159, 163, 165, 177, 184]
1: [7, 12, 20, 21, 25, 28, 29, 36, 42, 43, 57, 60, 64, 67, 69, 72, 74, 75, 78, 80, 91, 103, 110, 111, 113, 114, 119, 123, 125, 131, 142, 146, 148, 149, 153, 159, 163, 165, 168, 177, 184]
3: [2, 4, 5, 7, 9, 11, 12, 13, 18, 20, 21, 22, 23, 25, 28, 29, 32, 36, 39, 42, 43, 44, 57, 60, 64, 67, 68, 69, 72, 74, 75, 78, 79, 80, 88, 91, 92, 93, 94, 95, 96, 97, 101, 103, 104, 108, 110, 111, 113, 114, 117, 118, 119, 120, 121, 123, 125, 131, 136, 142, 146, 147, 148, 149, 153, 154, 157, 159, 160, 163, 164, 165, 168, 175, 177, 184, 185]
NA: [6, 15, 24, 34, 35, 47, 49, 52, 54, 63, 65, 66, 71, 89, 99, 100, 106, 107, 122, 126, 128, 129, 139, 141, 145, 151, 166, 169, 170, 172, 173, 174, 178, 179, 180, 181, 182, 187, 190]


In [42]:
CC = testing_2019.copy()
CC['Prediction'] = np.round(predictions)
CC['Difference'] = CC.SixthYear_HR - CC.Prediction

In [43]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = DDD.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.1099
plus or minus 1：0.2147
plus or minus 3：0.3979
plus or minus 5：0.5079
plus or minus 10：0.7435


In [44]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.972677904897959
mean_squared_error: 90.18605265621069
rmse: 9.496633753926213


In [45]:
Counter(sorted(p))

Counter({2.0: 5,
         9.0: 3,
         6.0: 2,
         7.0: 2,
         12.0: 2,
         13.0: 2,
         5.0: 1,
         8.0: 1,
         18.0: 1,
         32.0: 1,
         35.0: 1})

In [46]:
Counter(sorted(n))

Counter({2.0: 6,
         7.0: 5,
         1.0: 4,
         3.0: 3,
         9.0: 3,
         12.0: 3,
         6.0: 2,
         13.0: 2,
         29.0: 2,
         4.0: 1,
         5.0: 1,
         8.0: 1,
         14.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1,
         22.0: 1,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [47]:
Counter(sorted(m))

Counter({12.0: 9,
         2.0: 6,
         5.0: 6,
         1.0: 5,
         3.0: 5,
         7.0: 5,
         9.0: 4,
         0.0: 3,
         6.0: 3,
         23.0: 3,
         8.0: 2,
         10.0: 2,
         13.0: 2,
         15.0: 2,
         20.0: 2,
         22.0: 2,
         29.0: 2,
         4.0: 1,
         11.0: 1,
         14.0: 1,
         16.0: 1,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         21.0: 1,
         24.0: 1,
         25.0: 1,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [48]:
Counter(sorted(q))

Counter({2.0: 4,
         6.0: 3,
         23.0: 3,
         24.0: 3,
         33.0: 3,
         1.0: 2,
         3.0: 2,
         17.0: 2,
         19.0: 2,
         21.0: 2,
         31.0: 2,
         34.0: 2,
         35.0: 2,
         8.0: 1,
         10.0: 1,
         12.0: 1,
         14.0: 1,
         15.0: 1,
         16.0: 1,
         18.0: 1,
         22.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         44.0: 1,
         48.0: 1,
         49.0: 1})

In [49]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [21, 22, 25, 55, 67, 83, 90, 94, 102, 111, 114, 118, 119, 130, 131, 136, 142, 147, 148, 171, 184]
1: [4, 10, 13, 18, 21, 22, 25, 42, 53, 55, 57, 59, 60, 67, 74, 78, 83, 88, 90, 91, 93, 94, 98, 102, 111, 114, 118, 119, 130, 131, 136, 142, 147, 148, 153, 157, 159, 167, 171, 177, 184]
3: [1, 2, 4, 5, 10, 13, 18, 21, 22, 25, 29, 32, 36, 37, 42, 45, 48, 53, 55, 57, 59, 60, 67, 69, 72, 74, 75, 77, 78, 80, 82, 83, 84, 88, 90, 91, 93, 94, 95, 96, 98, 102, 104, 108, 110, 111, 114, 118, 119, 121, 125, 127, 130, 131, 134, 136, 137, 140, 142, 146, 147, 148, 149, 150, 152, 153, 157, 159, 160, 163, 167, 168, 171, 175, 177, 184]
NA: [6, 15, 16, 24, 31, 34, 41, 46, 47, 50, 52, 54, 58, 61, 63, 65, 66, 71, 79, 86, 89, 99, 100, 105, 106, 107, 115, 126, 128, 129, 139, 141, 144, 145, 151, 155, 162, 166, 169, 170, 172, 173, 174, 176, 180, 181, 182, 187, 188]


In [50]:
DD = testing_2019.copy()
DD['Prediction'] = np.round(predictions)
DD['Difference'] = DD.SixthYear_HR - DD.Prediction

In [51]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = EEE.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.1152
plus or minus1：0.2147
plus or minus 3：0.3822
plus or minus 5：0.5183
plus or minus 10：0.8010


In [52]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.57148983341237
mean_squared_error: 79.37487857303284
rmse: 8.909258025954397


In [53]:
Counter(sorted(p))

Counter({12.0: 4,
         3.0: 2,
         7.0: 2,
         11.0: 2,
         16.0: 2,
         2.0: 1,
         4.0: 1,
         5.0: 1,
         6.0: 1,
         10.0: 1,
         13.0: 1,
         15.0: 1,
         21.0: 1,
         23.0: 1,
         29.0: 1})

In [54]:
Counter(sorted(n))

Counter({12.0: 7,
         2.0: 4,
         9.0: 3,
         3.0: 2,
         5.0: 2,
         6.0: 2,
         7.0: 2,
         8.0: 2,
         10.0: 2,
         11.0: 2,
         13.0: 2,
         15.0: 2,
         16.0: 2,
         0.0: 1,
         1.0: 1,
         4.0: 1,
         17.0: 1,
         21.0: 1,
         23.0: 1,
         29.0: 1})

In [55]:
Counter(sorted(m))

Counter({12.0: 10,
         2.0: 6,
         5.0: 5,
         9.0: 5,
         1.0: 4,
         7.0: 4,
         0.0: 3,
         3.0: 3,
         6.0: 3,
         11.0: 3,
         13.0: 3,
         15.0: 3,
         17.0: 3,
         8.0: 2,
         10.0: 2,
         14.0: 2,
         16.0: 2,
         18.0: 2,
         23.0: 2,
         4.0: 1,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         27.0: 1,
         29.0: 1})

In [56]:
Counter(sorted(q))

Counter({2.0: 5,
         24.0: 3,
         33.0: 3,
         1.0: 2,
         3.0: 2,
         21.0: 2,
         26.0: 2,
         31.0: 2,
         34.0: 2,
         35.0: 2,
         16.0: 1,
         17.0: 1,
         23.0: 1,
         27.0: 1,
         28.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         41.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [57]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [7, 10, 21, 23, 36, 57, 60, 67, 74, 77, 91, 97, 108, 113, 118, 121, 130, 152, 159, 165, 177, 183]
1: [7, 9, 10, 12, 21, 23, 25, 29, 36, 42, 50, 55, 57, 60, 67, 69, 74, 75, 77, 80, 91, 95, 97, 108, 110, 113, 114, 118, 119, 121, 130, 142, 146, 148, 152, 159, 165, 175, 177, 183, 184]
3: [2, 4, 7, 9, 10, 12, 18, 20, 21, 22, 23, 25, 29, 33, 36, 42, 48, 50, 55, 57, 60, 67, 69, 72, 73, 74, 75, 77, 78, 80, 82, 87, 88, 90, 91, 93, 94, 95, 96, 97, 98, 102, 104, 108, 110, 113, 114, 118, 119, 121, 130, 131, 137, 140, 142, 146, 148, 152, 153, 154, 157, 158, 159, 160, 164, 165, 167, 171, 175, 177, 183, 184, 185]
NA: [6, 15, 19, 24, 34, 38, 47, 54, 61, 63, 65, 66, 71, 79, 81, 89, 99, 100, 106, 107, 126, 128, 129, 141, 144, 151, 155, 162, 166, 169, 170, 172, 173, 174, 179, 180, 182, 187]


In [58]:
EE = testing_2019.copy()
EE['Prediction'] = np.round(predictions)
EE['Difference'] = EE.SixthYear_HR - EE.Prediction

In [59]:
BB[(BB.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [60]:
DD[(DD.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [61]:
EE[(EE.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [62]:
AA[(AA.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [63]:
CC[(CC.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference
